In [1]:
%gui asyncio

In [2]:
import asyncio
import pandas as pd
import numpy as np

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [15]:
from ipywidgets import widgets as w
from IPython.display import clear_output
import time 



def annotate(df):
    cur_topic = 'Sustainable diets are healty'
    reset_opt = ''
    argumentative = w.ToggleButtons(
        options=[reset_opt, 0, 1, 'lacks context'],
        description='Argumentative:',
        disabled=False,
        button_style='',
    )
    
    evidence = w.ToggleButtons(
        options=[0, 1],
        description='Evidence:',
        disabled=False,
        button_style='',
    )
    
    claim = w.ToggleButtons(
        options=[0, 1],
        description='Claim:',
        disabled=False,
        button_style='',
    )
    
    procon = w.ToggleButtons(
        options=[-1, 0, 1],
        description='Pro/Con:',
        disabled=False,
        button_style='',
    )
    
    arg_type = w.ToggleButtons(
        options=[reset_opt, 'implicit', 'explicit', 'unrelated'],
        description='Argument type:',
        disabled=False,
        button_style='',
    )

    evidence_type = w.ToggleButtons(
        options=[reset_opt, 'expert', 'anecdotal', 'study'],
        description='Evidence type:',
        disabled=False,
        button_style='',
    )
    
    out = w.HTML()

    
    res = []
    async def f():
        for i, row in enumerate(df.iloc):
            t = "{:<307}".format(row.tweet)
            out.value = f'<div style="height: 50px;">#Tweet-num: {i+1}/{len(df)}<br/><strong>Topic:</strong><pref> {cur_topic}</pre><br/><strong>Tweet:</strong>{row.tweet.ljust(307)}</div>'
            av = await wait_for_change(argumentative, 'value')
            if av == 1:
                while evidence.value == '' or claim.value == '' or procon.value == '' or arg_type.value == '': 
                    at = await wait_for_change(arg_type, 'value')

            df.at[i, 'evidence'] = evidence.value
            df.at[i, 'claim'] = claim.value
            df.at[i, 'procon'] = procon.value
            df.at[i, 'arg_type'] = arg_type.value
            df.at[i, 'argumentative'] = argumentative.value
            df.at[i, 'topic'] = cur_topic
            df.at[i, 'evidence_type'] = evidence_type.value
            argumentative.value = reset_opt
            evidence.value = 0
            claim.value = 0
            procon.value = 0
            arg_type.value = reset_opt
            evidence_type.value = reset_opt

    asyncio.ensure_future(f())
    return w.AppLayout(header=out, footer=w.Box(children=[argumentative, claim, evidence, procon, evidence_type, arg_type])), res

In [16]:
df = pd.read_csv('filtered_tweets.csv')
test = pd.read_csv('test.csv')
dev = pd.read_csv('dev.csv')
df = df.loc[~df.tweet.isin(test['tweet'].values)]
df = df.loc[~df.tweet.isin(dev['tweet'].values)]#.sample(100)

In [17]:
sLength = len(df)
df = df.assign(argumentative=pd.Series(['']*sLength).values)
df = df.assign(evidence=pd.Series(np.zeros(sLength)).values)
df = df.assign(claim=pd.Series(np.zeros(sLength)).values)
df = df.assign(procon=pd.Series(np.zeros(sLength)).values)
df = df.assign(arg_type=pd.Series(['']*sLength).values)
df = df.assign(evidence_type=pd.Series(['']*sLength).values)
df = df.reset_index()

In [18]:
view, res = annotate(df)

In [19]:
view

AppLayout(children=(HTML(value='<div style="height: 50px;">#Tweet-num: 1/280<br/><strong>Topic:</strong><pref>…

In [25]:
df.to_csv('filtered_annotated.csv', index=False)